# Define Model(s)

---

In [5]:
# ##### DEGUBBING MENU:


# trial_run = True
# N_train_size= 5
# train_test_ratio = .5
# N_Monte_Carlo_Samples = 100
# # # Random DNN
# # f_unknown_mode = "Heteroskedastic_NonLinear_Regression"

# # # Random DNN internal noise
# # # f_unknown_mode = "DNN_with_Random_Weights"
# Depth_Bayesian_DNN = 2
# width = 50

# # # Random Dropout applied to trained DNN
# # f_unknown_mode = "DNN_with_Bayesian_Dropout"
# Dropout_rate = 0.1

# # GD with Randomized Input
# # f_unknown_mode = "GD_with_randomized_input"
# GD_epochs = 100

# # SDE with fractional Driver
# f_unknown_mode = "Rough_SDE"
# N_Euler_Steps = 10**1
# Hurst_Exponent = 0.5
# problem_dim = 3

# # Hyper-parameters of Cover
# delta = 0.01
# Proportion_per_cluster = .75

# # %run Loader.ipynb
# exec(open('Loader.py').read())
# # Load Packages/Modules
# exec(open('Init_Dump.py').read())
# import time as time #<- Note sure why...but its always seems to need 'its own special loading...'

# # %run Data_Simulator_and_Parser.ipynb
# exec(open('Data_Simulator_and_Parser.py').read())

# print("------------------------------")
# print("Running script for main model!")
# print("------------------------------")
# # %run Universal_Measure_Valued_Networks_Backend.ipynb
# exec(open('Universal_Measure_Valued_Networks_Backend.py').read())

# print("------------------------------------")
# print("Done: Running script for main model!")
# print("------------------------------------")

 40%|████      | 2/5 [00:00<00:00, 15.20it/s]

Deep Feature Builder - Ready
Deep Classifier - Ready
Deep Feature Builder - Ready
---------------------------------------
Beginning Data-Parsing/Simulation Phase
---------------------------------------
Deciding on Which Simulator/Parser To Load
Setting/Defining: Internal Parameters
Deciding on Which Type of Data to Get/Simulate
Simulating Output Data for given input data


100%|██████████| 38/38 [00:00<00:00, 23260.88it/s]


----------------------------------
Done Data-Parsing/Simulation Phase
----------------------------------
------------------------------
Running script for main model!
------------------------------
Deep Feature Builder - Ready
Deep Classifier - Ready
Training Classifer Portion of Type-A Model
Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    4.5s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    4.5s finished


...

Training Classifer Portion of Type Model: Done!
#--------------------#
 Get Training Error(s)
#--------------------#


  5%|▌         | 1/20 [00:00<00:01,  9.81it/s]

#-------------------------#
 Get Training Error(s): END
#-------------------------#
#----------------#
 Get Test Error(s)
#----------------#


100%|██████████| 20/20 [00:02<00:00,  9.21it/s]

#------------------------#
 Get Testing Error(s): END
#------------------------#
                                      DNM  MC-Oracle
W1-95L                           2.277640   0.000000
W1                               2.699185   0.000000
W1-95R                           3.145292   0.000000
M-95L                            2.308499   2.242574
M                                2.886287   2.886287
M-95R                            3.530251   3.445418
N_Par                          468.000000   0.000000
Train_Time                      10.952003   0.459962
Test_Time/MC-Oracle_Test_Time    0.382038   1.000000
------------------------------------
Done: Running script for main model!
------------------------------------


## Elastic Net Regressor

In [6]:
#=====================#
# Elastic Net Version #
#=====================#
# Block warnings that spam when performing coordinate descent (by default) in 1-d.
import warnings
from sklearn.linear_model import ElasticNetCV
warnings.filterwarnings("ignore")
# Initialize Elastic Net Regularization Model
ENET_reg = ElasticNetCV(cv=5, random_state=0, alphas = np.linspace(0,(10**2),(10**2)),
                           l1_ratio=np.linspace(0,1,(10**2)))

## ffNN Builder

In [7]:
def get_ffNN(height, depth, learning_rate, input_dim, output_dim):
    #----------------------------#
    # Maximally Interacting Layer #
    #-----------------------------#
    # Initialize Inputs
    input_layer = tf.keras.Input(shape=(input_dim,))
   
    
    #------------------#
    #   Core Layers    #
    #------------------#
    core_layers = fullyConnected_Dense(height)(input_layer)
    # Activation
    core_layers = tf.nn.swish(core_layers)
    # Train additional Depth?
    if depth>1:
        # Add additional deep layer(s)
        for depth_i in range(1,depth):
            core_layers = fullyConnected_Dense(height)(core_layers)
            # Activation
            core_layers = tf.nn.swish(core_layers)
    
    #------------------#
    #  Readout Layers  #
    #------------------# 
    # Affine (Readout) Layer (Dense Fully Connected)
    output_layers = fullyConnected_Dense(output_dim)(core_layers)  
    # Define Input/Output Relationship (Arch.)
    trainable_layers_model = tf.keras.Model(input_layer, output_layers)
    
    
    #----------------------------------#
    # Define Optimizer & Compile Archs.
    #----------------------------------#
    opt = Adam(lr=learning_rate)
    trainable_layers_model.compile(optimizer=opt, loss="mae", metrics=["mse", "mae", "mape"])

    return trainable_layers_model



def build_ffNN(n_folds , n_jobs, n_iter, param_grid_in, X_train, y_train,X_test):
    # Update Dictionary
    param_grid_in_internal = param_grid_in
    param_grid_in_internal['input_dim'] = [(X_train.shape[1])]
    
    # Deep Feature Network
    ffNN_CV = tf.keras.wrappers.scikit_learn.KerasRegressor(build_fn=get_ffNN, 
                                                            verbose=True)
    
    # Randomized CV
    ffNN_CVer = RandomizedSearchCV(estimator=ffNN_CV, 
                                    n_jobs=n_jobs,
                                    cv=KFold(n_folds, random_state=2020, shuffle=True),
                                    param_distributions=param_grid_in_internal,
                                    n_iter=n_iter,
                                    return_train_score=True,
                                    random_state=2020,
                                    verbose=10)
    
    # Fit Model #
    #-----------#
    ffNN_CVer.fit(X_train,y_train)

    # Write Predictions #
    #-------------------#
    y_hat_train = ffNN_CVer.predict(X_train)
    
    eval_time_ffNN = time.time()
    y_hat_test = ffNN_CVer.predict(X_test)
    eval_time_ffNN = time.time() - eval_time_ffNN
    
    # Counter number of parameters #
    #------------------------------#
    # Extract Best Model
    best_model = ffNN_CVer.best_estimator_
    # Count Number of Parameters
    N_params_best_ffNN = np.sum([np.prod(v.get_shape().as_list()) for v in best_model.model.trainable_variables])
    
    
    # Return Values #
    #---------------#
    return y_hat_train, y_hat_test, N_params_best_ffNN, eval_time_ffNN

# Update User
#-------------#
print('Deep Feature Builder - Ready')


Deep Feature Builder - Ready


# Gradient-Boosted Random Forest Regressor

In [8]:
def get_GBRF(X_train,X_test,y_train):

    # Run Random Forest Util
    rand_forest_model_grad_boosted = GradientBoostingRegressor()

    # Grid-Search CV
    Random_Forest_GridSearch = RandomizedSearchCV(estimator = rand_forest_model_grad_boosted,
                                                  n_iter=n_iter_trees,
                                                  cv=KFold(CV_folds, random_state=2020, shuffle=True),
                                                  param_distributions=Rand_Forest_Grid,
                                                  return_train_score=True,
                                                  random_state=2020,
                                                  verbose=10,
                                                  n_jobs=n_jobs)

    random_forest_trained = Random_Forest_GridSearch.fit(X_train,y_train)
    random_forest_trained = random_forest_trained.best_estimator_

    #--------------------------------------------------#
    # Write: Model, Results, and Best Hyper-Parameters #
    #--------------------------------------------------#

    # Save Model
    # pickle.dump(random_forest_trained, open('./outputs/models/Gradient_Boosted_Tree/Gradient_Boosted_Tree_Best.pkl','wb'))

    # Save Readings
    cur_path = os.path.expanduser('./outputs/tables/best_params_Gradient_Boosted_Tree.txt')
    with open(cur_path, "w") as f:
        f.write(str(Random_Forest_GridSearch.best_params_))

    best_params_table_tree = pd.DataFrame({'N Estimators': [Random_Forest_GridSearch.best_params_['n_estimators']],
                                        'Min Samples Leaf': [Random_Forest_GridSearch.best_params_['min_samples_leaf']],
                                        'Learning Rate': [Random_Forest_GridSearch.best_params_['learning_rate']],
                                        'Max Depth': [Random_Forest_GridSearch.best_params_['max_depth']],
                                        })
    
    # Count Number of Parameters in Random Forest Regressor
    N_tot_params_per_tree = [ (x[0].tree_.node_count)*random_forest_trained.n_features_ for x in random_forest_trained.estimators_]
    N_tot_params_in_forest = sum(N_tot_params_per_tree)
    best_params_table_tree['N_parameters'] = [N_tot_params_in_forest]
    # Write Best Parameter(s)
    best_params_table_tree.to_latex('./outputs/tables/Best_params_table_Gradient_Boosted_Tree.txt')
    #---------------------------------------------#
    
    # Generate Prediction(s) #
    #------------------------#
    y_train_hat_random_forest_Gradient_boosting = random_forest_trained.predict(X_train)
    eval_time_GBRF = time.time()
    y_test_hat_random_forest_Gradient_boosting = random_forest_trained.predict(X_test)
    eval_time_GBRF = time.time() - eval_time_GBRF
    
    # Return Predictions #
    #--------------------#
    return y_train_hat_random_forest_Gradient_boosting, y_test_hat_random_forest_Gradient_boosting, random_forest_trained, N_tot_params_in_forest, eval_time_GBRF

## Kernel Ridge Regressor

In [9]:
def get_Kernel_Ridge_Regressor(data_x_in,data_x_test_in,data_y_in):
    # Imports
    from sklearn.svm import SVR
    from sklearn.kernel_ridge import KernelRidge

    # Initialize Randomized Gridsearch
    kernel_ridge_CVer = RandomizedSearchCV(estimator = KernelRidge(),
                                           n_jobs=n_jobs,
                                           cv=KFold(CV_folds, random_state=2020, shuffle=True),
                                           param_distributions=param_grid_kernel_Ridge,
                                           n_iter=n_iter,
                                           return_train_score=True,
                                           random_state=2020,
                                           verbose=10)
    kernel_ridge_CVer.fit(data_x_in,data_y_in)

    # Get best Kernel ridge regressor
    best_kernel_ridge_model = kernel_ridge_CVer.best_estimator_

    # Get Predictions
    f_hat_kernel_ridge_train = best_kernel_ridge_model.predict(data_x_in)
    eval_time_kr = time.time()
    f_hat_kernel_ridge_test = best_kernel_ridge_model.predict(data_x_test_in)
    eval_time_kr = time.time() - eval_time_kr

    # Count Parameters of best model
    N_params_kR = len(best_kernel_ridge_model.get_params()) + 2*problem_dim
    
    Path('./outputs/models/Kernel_Ridge/').mkdir(parents=True, exist_ok=True)
    pd.DataFrame.from_dict(kernel_ridge_CVer.best_params_,orient='index').to_latex("./outputs/models/Kernel_Ridge/Best_Parameters.tex")
    
    
    
    # Return
    return f_hat_kernel_ridge_train, f_hat_kernel_ridge_test, best_kernel_ridge_model, N_params_kR, eval_time_kr

---

# Train Models and Get Prediction(s)

### Elastic Net

In [169]:
# Start Timer
Timer_ENET = time.time()

print("--------------")
print("Training: ENET")
print("--------------")

if output_dim == 1:
    # Fit Elastic Net Model
    ENET_reg.fit(X_train,Y_train_mean_emp)

    # Get Predictions
    ENET_predict = ENET_reg.predict(X_train)
    ENET_eval_time = time.time()
    ENET_predict_test = ENET_reg.predict(X_test)
    ENET_eval_time = time.time() - ENET_eval_time
    
else:
    for d in tqdm(range(output_dim)):
        # Fit Elastic Net Model
        ENET_reg.fit(X_train,Y_train_mean_emp[:,d])

        # Get Predictions
        ENET_predict_loop = ENET_reg.predict(X_train)
        ENET_eval_time_loop = time.time()
        ENET_predict_loop_test = ENET_reg.predict(X_test)
        ENET_eval_time_loop = time.time() - ENET_eval_time_loop
    
        if d == 0:
            ENET_predict_train = ENET_predict_loop.reshape(-1,1)
            ENET_predict_test = ENET_predict_loop_test.reshape(-1,1)
            ENET_N_Params = X_train.shape[0]*2
            ENET_eval_time = ENET_eval_time_loop
        else:
            ENET_predict_train = np.append(ENET_predict_train,ENET_predict_loop.reshape(-1,1),axis=1)
            ENET_predict_test = np.append(ENET_predict_test,ENET_predict_loop_test.reshape(-1,1),axis=1)
            ENET_N_Params = ENET_N_Params + X_train.shape[0]*2
            ENET_eval_time = ENET_eval_time + ENET_eval_time_loop

# Stop Timer
Timer_ENET = time.time() - Timer_ENET
print("---------------------")
print("Training: ENET - Done")
print("---------------------")

  0%|          | 0/3 [00:00<?, ?it/s]

--------------
Training: ENET
--------------


100%|██████████| 3/3 [00:16<00:00,  5.41s/it]

---------------------
Training: ENET - Done
---------------------


#### Evaluate Prediction Quality

In [171]:
## Train
ENET_errors_W1, ENET_errors_M1 = get_deterministic_errors(X_train,ENET_predict_train,Y_train,N_Bootstraps=N_Boostraps_BCA)
## Test
ENET_errors_W1_test, ENET_errors_M1_test = get_deterministic_errors(X_test,ENET_predict_test,Y_test,N_Bootstraps=N_Boostraps_BCA)
# Stop Timer
Timer_ENET = time.time() - Timer_ENET

 20%|██        | 10/50 [00:00<00:00, 95.39it/s]

#------------#
 Get Error(s) 
#------------#


100%|██████████| 20/20 [00:00<00:00, 137.47it/s]

#-----------------#
 Get Error(s): END 
#-----------------#
#------------#
 Get Error(s) 
#------------#
#-----------------#
 Get Error(s): END 
#-----------------#


#### Update DataFrame

In [172]:
# Train
Summary_pred_Qual_models["ENET"] = pd.Series(np.append(np.append(ENET_errors_W1,
                                                                ENET_errors_M1),
                                                         np.array([ENET_N_Params,
                                                                   Timer_ENET,
                                                                   (ENET_eval_time/Test_Set_PredictionTime_MC)])), index=Summary_pred_Qual_models.index)
# Test
Summary_pred_Qual_models_test["ENET"] = pd.Series(np.append(np.append(ENET_errors_W1_test,
                                                                     ENET_errors_M1_test),
                                                           np.array([ENET_N_Params,
                                                                     Timer_ENET,
                                                                     (ENET_eval_time/Test_Set_PredictionTime_MC)])), index=Summary_pred_Qual_models_test.index)

print("Updated DataFrame")
print(Summary_pred_Qual_models_test)
Summary_pred_Qual_models_test

Updated DataFrame
                                      DNM  MC-Oracle         DNN      KRidge  \
W1-95L                           2.277640   0.000000    2.704290  192.530168   
W1                               2.699185   0.000000    3.518672  249.420563   
W1-95R                           3.145292   0.000000    4.161309  307.297392   
M-95L                            2.308499   2.242574    5.497128  559.465667   
M                                2.886287   2.886287    6.780059  710.707455   
M-95R                            3.530251   3.445418    8.426148  808.223948   
N_Par                          468.000000   0.000000   83.000000    0.000000   
Train_Time                      10.952003   0.459962  216.869673  581.567460   
Test_Time/MC-Oracle_Test_Time    0.382038   1.000000    0.566234    0.004881   

                                       GBRF          ENET  
W1-95L                         1.944196e+02  1.964914e+02  
W1                             2.501461e+02  2.537312e+02  
W

,DNM,MC-Oracle,DNN,KRidge,GBRF,ENET
W1-95L,2.277640,0.000000,2.704290,192.530168,1.944196e+02,1.964914e+02
W1,2.699185,0.000000,3.518672,249.420563,2.501461e+02,2.537312e+02
W1-95R,3.145292,0.000000,4.161309,307.297392,3.203476e+02,2.886787e+02
M-95L,2.308499,2.242574,5.497128,559.465667,6.108610e+02,5.998184e+02
M,2.886287,2.886287,6.780059,710.707455,7.170442e+02,7.228298e+02
M-95R,3.530251,3.445418,8.426148,808.223948,8.975201e+02,8.616056e+02
N_Par,468.000000,0.000000,83.000000,0.000000,1.698080e+05,3.000000e+02
Train_Time,10.952003,0.459962,216.869673,581.567460,1.619748e+09,1.619748e+09
Test_Time/MC-Oracle_Test_Time,0.382038,1.000000,0.566234,0.004881,2.524703e-02,1.544312e-03


## Kernel Ridge Regression

In [116]:
# %run Evaluation.ipynb
exec(open('Evaluation.py').read())

In [117]:
# Stop Timer
Timer_kRidge = time.time()


print("-----------------")
print("Training: K-Ridge")
print("-----------------")

Xhat_Kridge, Xhat_Kridge_test , relic, kRidge_N_params, KRidge_eval_time = get_Kernel_Ridge_Regressor(X_train,X_test,Y_train_mean_emp)

-----------------
Training: K-Ridge
-----------------
Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Batch computation too fast (0.0201s.) Setting batch_size=2.
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.4s finished


#### Get Prediction Errors

In [137]:
## Train
kRidge_errors_W1, kRidge_errors_M1 = get_deterministic_errors(X_train,Xhat_Kridge,Y_train,N_Bootstraps=N_Boostraps_BCA)
## Test
kRidge_errors_W1_test, kRidge_errors_M1_test = get_deterministic_errors(X_test,Xhat_Kridge_test,Y_test,N_Bootstraps=N_Boostraps_BCA)
# Stop Timer
Timer_kRidge = time.time() - Timer_kRidge

 24%|██▍       | 12/50 [00:00<00:00, 119.32it/s]

#------------#
 Get Error(s) 
#------------#


100%|██████████| 20/20 [00:00<00:00, 142.88it/s]

#-----------------#
 Get Error(s): END 
#-----------------#
#------------#
 Get Error(s) 
#------------#
#-----------------#
 Get Error(s): END 
#-----------------#


#### Update DataFrame

In [138]:
# Train
Summary_pred_Qual_models["KRidge"] = pd.Series(np.append(np.append(kRidge_errors_W1,
                                                                kRidge_errors_M1),
                                                         np.array([0,
                                                                   Timer_kRidge,
                                                                   (KRidge_eval_time/Test_Set_PredictionTime_MC)])), index=Summary_pred_Qual_models.index)
# Test
Summary_pred_Qual_models_test["KRidge"] = pd.Series(np.append(np.append(kRidge_errors_W1_test,
                                                                     kRidge_errors_M1_test),
                                                           np.array([0,
                                                                     Timer_kRidge,
                                                                     (KRidge_eval_time/Test_Set_PredictionTime_MC)])), index=Summary_pred_Qual_models_test.index)

print("Updated DataFrame")
print(Summary_pred_Qual_models_test)
Summary_pred_Qual_models_test

Updated DataFrame
                                      DNM  MC-Oracle         DNN      KRidge
W1-95L                           2.277640   0.000000    2.704290  192.530168
W1                               2.699185   0.000000    3.518672  249.420563
W1-95R                           3.145292   0.000000    4.161309  307.297392
M-95L                            2.308499   2.242574    5.497128  559.465667
M                                2.886287   2.886287    6.780059  710.707455
M-95R                            3.530251   3.445418    8.426148  808.223948
N_Par                          468.000000   0.000000   83.000000    0.000000
Train_Time                      10.952003   0.459962  216.869673  581.567460
Test_Time/MC-Oracle_Test_Time    0.382038   1.000000    0.566234    0.004881


## Gradient-Boosted Random Forest

In [162]:
# Start Timer
Timer_GBRF = time.time()

print("--------------")
print("Training: GBRF")
print("--------------")

if output_dim == 1:
    GBRF_y_hat_train, GBRF_y_hat_test, GBRF_model, GBRF_N_Params, GBRF_eval_time = get_GBRF(X_train,
                                                                                            X_test,
                                                                                            Y_train_mean_emp)
else:
    for d in range(output_dim):
        GBRF_y_hat_train_loop, GBRF_y_hat_test_loop, GBRF_model, GBRF_N_Params_loop, GBRF_eval_time_loop = get_GBRF(X_train,
                                                                                                                X_test,
                                                                                                                Y_train_mean_emp[:,d])
        if d == 0:
            GBRF_y_hat_train = GBRF_y_hat_train_loop.reshape(-1,1)
            GBRF_y_hat_test = GBRF_y_hat_test_loop.reshape(-1,1)
            GBRF_N_Params = GBRF_N_Params_loop
            GBRF_eval_tim = GBRF_eval_time_loop
        else:
            GBRF_y_hat_train = np.append(GBRF_y_hat_train,GBRF_y_hat_train_loop.reshape(-1,1),axis=1)
            GBRF_y_hat_test = np.append(GBRF_y_hat_test,GBRF_y_hat_test_loop.reshape(-1,1),axis=1)
            GBRF_N_Params = GBRF_N_Params + GBRF_N_Params_loop
            GBRF_eval_tim = GBRF_eval_tim + GBRF_eval_time_loop

--------------
Training: GBRF
--------------
Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=4)]: Batch computation too fast (0.1969s.) Setting batch_size=2.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=4)]: Batch computation too fast (0.1581s.) Setting batch_size=2.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    0.2s finished


#### Get Prediction Errors

In [164]:
## Train
GBRF_errors_W1, GBRF_errors_M1 = get_deterministic_errors(X_train,GBRF_y_hat_train,Y_train,N_Bootstraps=N_Boostraps_BCA)
## Test
GBRF_errors_W1_test, GBRF_errors_M1_test = get_deterministic_errors(X_test,GBRF_y_hat_test,Y_test,N_Bootstraps=N_Boostraps_BCA)


# Compute Lapsed Time Needed For Training
Timer_GBRF = time.time() - Timer_GBRF

 22%|██▏       | 11/50 [00:00<00:00, 106.18it/s]

#------------#
 Get Error(s) 
#------------#


100%|██████████| 20/20 [00:00<00:00, 159.52it/s]

#-----------------#
 Get Error(s): END 
#-----------------#
#------------#
 Get Error(s) 
#------------#
#-----------------#
 Get Error(s): END 
#-----------------#


#### Update Dataframe

In [165]:
# Train
Summary_pred_Qual_models["GBRF"] = pd.Series(np.append(np.append(GBRF_errors_W1,
                                                                GBRF_errors_M1),
                                                         np.array([GBRF_N_Params,
                                                                   Timer_GBRF,
                                                                   (GBRF_eval_tim/Test_Set_PredictionTime_MC)])), index=Summary_pred_Qual_models.index)
# Test
Summary_pred_Qual_models_test["GBRF"] = pd.Series(np.append(np.append(GBRF_errors_W1_test,
                                                                     GBRF_errors_M1_test),
                                                           np.array([GBRF_N_Params,
                                                                     Timer_GBRF,
                                                                     (GBRF_eval_tim/Test_Set_PredictionTime_MC)])), index=Summary_pred_Qual_models_test.index)

print("Updated DataFrame")
print(Summary_pred_Qual_models_test)
Summary_pred_Qual_models_test

Updated DataFrame
                                      DNM  MC-Oracle         DNN      KRidge  \
W1-95L                           2.277640   0.000000    2.704290  192.530168   
W1                               2.699185   0.000000    3.518672  249.420563   
W1-95R                           3.145292   0.000000    4.161309  307.297392   
M-95L                            2.308499   2.242574    5.497128  559.465667   
M                                2.886287   2.886287    6.780059  710.707455   
M-95R                            3.530251   3.445418    8.426148  808.223948   
N_Par                          468.000000   0.000000   83.000000    0.000000   
Train_Time                      10.952003   0.459962  216.869673  581.567460   
Test_Time/MC-Oracle_Test_Time    0.382038   1.000000    0.566234    0.004881   

                                       GBRF  
W1-95L                         1.944196e+02  
W1                             2.501461e+02  
W1-95R                         3.203476e+02

,DNM,MC-Oracle,DNN,KRidge,GBRF
W1-95L,2.277640,0.000000,2.704290,192.530168,1.944196e+02
W1,2.699185,0.000000,3.518672,249.420563,2.501461e+02
W1-95R,3.145292,0.000000,4.161309,307.297392,3.203476e+02
M-95L,2.308499,2.242574,5.497128,559.465667,6.108610e+02
M,2.886287,2.886287,6.780059,710.707455,7.170442e+02
M-95R,3.530251,3.445418,8.426148,808.223948,8.975201e+02
N_Par,468.000000,0.000000,83.000000,0.000000,1.698080e+05
Train_Time,10.952003,0.459962,216.869673,581.567460,1.619748e+09
Test_Time/MC-Oracle_Test_Time,0.382038,1.000000,0.566234,0.004881,2.524703e-02


## Feed-Forward DNN

In [129]:
# Stop Timer
Timer_ffNN = time.time()
print("-------------")
print("Training: DNN")
print("-------------")

# Redefine (Dimension-related) Elements of Grid
param_grid_Deep_Classifier['input_dim'] = [problem_dim]
param_grid_Deep_Classifier['output_dim'] = [output_dim]

YHat_ffNN, YHat_ffNN_test, ffNN_N_Params, ffNN_eval_time = build_ffNN(n_folds = CV_folds,
                                                                      n_jobs = n_jobs, 
                                                                      n_iter = n_iter,
                                                                      param_grid_in = param_grid_Deep_Classifier,  
                                                                      X_train = X_train,
                                                                      y_train = Y_train_mean_emp,
                                                                      X_test = X_test)

-------------
Training: DNN
-------------
Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    4.9s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    4.9s finished


Epoch 1/5
7/7 [==============================] - 0s 1ms/step - loss: 251.1948 - mse: 87408.2734 - mae: 251.1948 - mape: 99.9685
Epoch 2/5
7/7 [==============================] - 0s 3ms/step - loss: 251.1946 - mse: 87408.2109 - mae: 251.1946 - mape: 99.9684
Epoch 3/5
7/7 [==============================] - 0s 2ms/step - loss: 251.1945 - mse: 87408.1328 - mae: 251.1945 - mape: 99.9683
Epoch 4/5
7/7 [==============================] - 0s 3ms/step - loss: 251.1943 - mse: 87408.0625 - mae: 251.1943 - mape: 99.9682
Epoch 5/5
3/3 [==============================] - 0s 947us/step


#### Get Prediction Errors

In [130]:
## Train
ffNN_errors_W1,ffNN_errors_M1 = get_deterministic_errors(X_train,YHat_ffNN,Y_train,N_Bootstraps=N_Boostraps_BCA)
## Test
ffNN_errors_W1_test,ffNN_errors_M1_test = get_deterministic_errors(X_test,YHat_ffNN_test,Y_test,N_Bootstraps=N_Boostraps_BCA)

# Compute Lapsed Time Needed For Training
Timer_ffNN =  time.time() - Timer_ffNN

 18%|█▊        | 9/50 [00:00<00:00, 84.68it/s]

#------------#
 Get Error(s) 
#------------#


100%|██████████| 20/20 [00:00<00:00, 153.18it/s]

#-----------------#
 Get Error(s): END 
#-----------------#
#------------#
 Get Error(s) 
#------------#
#-----------------#
 Get Error(s): END 
#-----------------#


#### Update DataFrame

In [139]:
# Train
Summary_pred_Qual_models["DNN"] = pd.Series(np.append(np.append(ffNN_errors_W1,
                                                                ffNN_errors_M1),
                                                         np.array([ffNN_N_Params,
                                                                   Timer_ffNN,
                                                                   (ffNN_eval_time/Test_Set_PredictionTime_MC)])), index=Summary_pred_Qual_models.index)
# Test
Summary_pred_Qual_models_test["DNN"] = pd.Series(np.append(np.append(ffNN_errors_W1_test,
                                                                     ffNN_errors_M1_test),
                                                           np.array([ffNN_N_Params,
                                                                     Timer_ffNN,
                                                                     (ffNN_eval_time/Test_Set_PredictionTime_MC)])), index=Summary_pred_Qual_models_test.index)

print("Updated DataFrame")
print(Summary_pred_Qual_models_test)
Summary_pred_Qual_models_test

Updated DataFrame
                                      DNM  MC-Oracle         DNN      KRidge
W1-95L                           2.277640   0.000000    2.704290  192.530168
W1                               2.699185   0.000000    3.518672  249.420563
W1-95R                           3.145292   0.000000    4.161309  307.297392
M-95L                            2.308499   2.242574    5.497128  559.465667
M                                2.886287   2.886287    6.780059  710.707455
M-95R                            3.530251   3.445418    8.426148  808.223948
N_Par                          468.000000   0.000000   83.000000    0.000000
Train_Time                      10.952003   0.459962  216.869673  581.567460
Test_Time/MC-Oracle_Test_Time    0.382038   1.000000    0.566234    0.004881


,DNM,MC-Oracle,DNN,KRidge
W1-95L,2.277640,0.000000,2.704290,192.530168
W1,2.699185,0.000000,3.518672,249.420563
W1-95R,3.145292,0.000000,4.161309,307.297392
M-95L,2.308499,2.242574,5.497128,559.465667
M,2.886287,2.886287,6.780059,710.707455
M-95R,3.530251,3.445418,8.426148,808.223948
N_Par,468.000000,0.000000,83.000000,0.000000
Train_Time,10.952003,0.459962,216.869673,581.567460
Test_Time/MC-Oracle_Test_Time,0.382038,1.000000,0.566234,0.004881


---